In [13]:
# ! pip install tiktoken
# ! pip install --upgrade tiktoken
# from tiktoken import Tokenizer

In [62]:
import os
import re
import ast
import pandas as pd
import json
from IPython.display import display, HTML, Code
from IPython.display import clear_output
# from tiktoken import Tokenizer, TokenCount
# from tiktoken.models import Model

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_row', 100)

C:\Users\N\AppData\Local\Temp\ipykernel_6204\3524316874.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [14]:
import Tweet_Geo_constants as constants
import helper
# import LLM_Geo_kernel.Solution as Solution

from Tweet_Geo_kernel import Localization

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load tweet user infomration

In [58]:
user_CSV = r'.\user_500.csv'
raw_user_df = pd.read_csv(user_CSV)

raw_user_df = raw_user_df.fillna("")
raw_user_df['description'] = raw_user_df['description'].apply(helper.clean_tweet)
raw_user_df['location'] = raw_user_df['location'].apply(helper.clean_tweet)


raw_user_df[['id', 'description', 'location']]

,id,description,location
0,33,Retired York Regional police officer living with a physical disability. I do not believe in fake science or government mandated drugs.,"Elliott Lake, Ontario, Canada"
1,34,Telling the other side of the story. Follow and,Canada
2,35,Speaker. Author My Tar Sands Tipping Point w CBC. Communications Mngr-,
3,36,I Can Fart On Command Community gardening is a tactic.,Desert Earth
4,37,Print and online journal dedicated to thoughtful journalism about environmental and other intersectional issues. Send pitches to submissions.org.,"ÜT: 37.7889,-122.401312"
...,...,...,...
536,569,,"Southern Corsica, Corsica"
537,570,Dartmouthian. Mother of Vincent. Multimedia journalist and producer . Personal Tweeter. Goose memorializer. Saver of bees & chairs. Totus Tuus.,"Ponamogoatitjg/ Dartmouth, NS"
538,571,"PhD, US history, JD. Book:","Oklahoma City, OK"
539,572,"Welcome to our support channel. Burning questions? Head to the Neighbourhood - Pour un soutien en français, visitez",Canada


In [71]:
user_df = raw_user_df.fillna("").sample(80)  # maximum = 80.

user_df['raw_id'] = user_df['id']

user_df['id'] = range(1, len(user_df) + 1, 1)

used_cols=['id', 'description', 'location']


# model=r"gpt-3.5-turbo-0613"
# model=r"gpt-3.5-turbo"
model = "gpt-4"

tweet_loc = Localization(user_df=user_df,
                        model=model,
                        max_tweet=80,
                        used_cols=['id', 'description', 'location'],
                        )

print(tweet_loc.prompt_for_user_localization)
print()
response_for_user_loc = tweet_loc.get_LLM_user_loc_response() 

content = helper.extract_content_from_LLM_reply(response_for_user_loc)
if content[-1] == ";":
    content = content[:-1]
# extracted_location_list = ast.literal_eval(content)
# extracted_location_list = content.split(";")

extracted_location_list = location_str_to_list(content)
extracted_loc_df = pd.DataFrame(extracted_location_list, columns=['id', 'extracted_location'])
extracted_loc_df['id'] = extracted_loc_df['id'].astype(int)

if len(user_df) != len(extracted_location_list):
    print("***************  ERROR  *****************")
    print(f"The extracted location count ({len(extracted_location_list)}) is not equal to the input user count ({len(user_df)})!")
    
else:
    print(f"The extracted location count ({len(extracted_location_list)}) is  equal to the input user count ({len(user_df)})!")
    # print("Extracted locations:")
    # print(extracted_location_list)
    
merged_df = user_df[used_cols].merge(extracted_loc_df, how='right', right_on='id', left_on='id') 
merged_df['id'] = user_df['raw_id'].to_list()

merged_df

Your role: A professional Geographer who is extracting location information from tweet texts. 
Your task: Extract Twitter user’s home location based on their profile description and self-reported home location.  
Requirement:
1. The input is a list of dictionary elements, e.g.: [{"id": 56332, "description": "Heart of the big apple.", "location:": "New York City"}, {"userid": 635, "description": "Writer, house wife.", "location:": "Rural area"}]
2. The output is a long string, the format is: "id1:location1;id2:location2;". E.g.: 56332:New York City, US;635:;. The ";" is used to separate the each user location; the "," is used to separate the location parts or levels. The output user order should be the same as the input, and the output count MUST equal to the input count.
3. Return the location string only, DO NOT add explanation and description.
4. The semicolon, ";", is not allow in the location name. Make sure the output element count is the same as the input. If you cannot find or d

,id,description,location,extracted_location
0,49,Fmr Hotshot Overhead/Crew Supervisor. Call it like it is. Subscribing to the Newsletter/Podcast supports firefighter donations and content👇,MN\MT,"MN,MT"
1,172,"Software Developer/Architect- OSDev, Embedded Systems, C/C++, Rust, x86-64/ARM Assembly, PAYPHONES. Often sarcastic; Court Reporter; #TransRightsAreHumanRights","Calgary, Alberta, Canada.","Calgary, Alberta, Canada"
2,267,Retired,Nova Scotia,"Nova Scotia, Canada"
3,561,"multi national(canadian, British, American) family . Born with a questioning mind. i have No time for bigots, racists, facists,or MeMe's NO DMs. MARRIED",,
4,125,Male,"Calgary, Alberta, Canada","Calgary, Alberta, Canada"
5,138,"News Curator, Content Curator",,
6,33,Retired York Regional police officer living with a physical disability. I do not believe in fake science or government mandated drugs.,"Elliott Lake, Ontario, Canada","Elliott Lake, Ontario, Canada"
7,278,Make equity and health the focus for safer communities and streets. Tweets by Martyn and maybe Ella & John too (personal views).,"Sipekne’katik, Mi'kma'ki","Sipekne’katik, Mi'kma'ki"
8,522,"Writer, editor. Publications: creative non-fiction, fiction (pen-name Goldsmith). Communications & media relations, freelance writer.","Oakland, California","Oakland, California, US"
9,187,,,
